<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/ways_of_convert_rubert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

Этот ноутбук для бекендеров, которым дали модельку и сказали деплоить так, чтобы она быстро работала. И больше ничего не дали, кроме модельки
	
  (・_・ヾ

Устанавливаем нужные зависимости

In [60]:
!pip install onnx transformers onnxruntime-gpu folium==0.2.1 optimum[onnxruntime]

ERROR: Could not find a version that satisfies the requirement torch_pruning==0.3.0 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.2.0, 0.2.1, 0.2.4, 0.2.5, 0.2.6, 0.2.7)
ERROR: No matching distribution found for torch_pruning==0.3.0


Фиксируем версии библиотек

In [6]:
!pip freeze > req.txt

Импорты

In [7]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from torch.nn.utils import prune
from optimum.onnxruntime import ORTQuantizer
from torch.onnx import TrainingMode
from onnxruntime.quantization import quantize_dynamic, QuantType

Качаем датасет, на котором будем проверять качество модели

In [8]:
!wget https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv

--2022-05-03 15:56:12--  https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv [following]
--2022-05-03 15:56:13--  https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21781685 (21M) [text/plain]
Saving to: ‘women-clothing-accessories.3-class.balanced.csv.3’

women-clothing-acce 100%[===================>]  20.77M  69.2MB/s   

# Об нейросети

В качестве подопытного будем использовать [rubert-base-cased-sentiment](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для классификации русских предложений. Данная нейросеть предсказывает 3 метки класса, в зависимости от тона предложения - позитивное, негативное или нейтральное

Запускаем нейросеть как есть

In [9]:
device  = torch.device('cuda') #будем все запускать на gpu

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True).to(device)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [11]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

Проверим время инференса модели

In [76]:
%%time
predict(text)

CPU times: user 26.4 ms, sys: 1.92 ms, total: 28.3 ms
Wall time: 30.5 ms


0

Проверим качество модели. Для проверки качества будем использовать один из датасетов, на котором обучалась модель, а именно [этот](https://github.com/sismetanin/rureviews)

In [13]:
df = pd.read_csv('/content/women-clothing-accessories.3-class.balanced.csv', delimiter='\t')
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


Для удобства немного изменим датасет - заменим метки класса на цифровые значения и выберем 1000 рандомных строк

In [14]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:1000]
mapping = {'negative': 2, 'positive': 1, 'neautral':0}
df = df.replace({'sentiment': mapping})
df.head()

,review,sentiment
0,"Заказывала второй раз, порождала 4 месяца, но ...",1
1,"Отличный бюстгальтер, держится хорошо, пришёл ...",1
2,"Материал ужасный, нитки торчат, короткая, еди...",2
3,"заказ не пришел,но деньги вернули",2
4,"Продавец заявил-высокая талия,материал-поплин ...",2


Считаем качество

In [15]:
texts = list(df['review'])
labels = list(df['sentiment'])

In [16]:
predictions = [predict(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7783219952095606, recall: 0.7634240759240759, f1score: 0.7558935727560435


Сохраним оригинальную модель и посмотрим на ее вес

In [17]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [18]:
torch.save(model, 'output/original.pt')

In [19]:
!du -shc output/original.pt

679M	output/original.pt
679M	total


# ONNX

Формат Open Neural Network Exchange (ONNX) обеспечит общий способ представления данных, используемых в нейронных сетях. Большинство платформ имеют сегодня собственный специфический формат моделей, которые способны работать с моделями других платформ только при использовании специальных инструментов преобразования форматов.

ONNX позволит осуществлять свободный обмен информацией, которой обладают модели, без процедуры преобразования. Модель, обученную на одной платформе, можно будет использовать и на другой платформе. Также можно будет модель, обученную на одном фреймворке, перенести на другой фреймворк.

Перевести модель в ONNX можно несколькими способами:

1. Есть способ конвертации модели через torch.onnx

In [20]:
!mkdir -p output/onnx_transforms

In [92]:
model.device

device(type='cuda', index=0)

In [93]:
#делаем dummy input
dummy_input0 = torch.randint(1, 224, (1,512)).to(device)
dummy_input1 = torch.randint(0, 1, (1,512 )).to(device)
dummy_input2 =  torch.randint(0, 1, (1,512 )).to(device)
dummy_inputs = (dummy_input0,dummy_input1,dummy_input2)
with torch.no_grad():
  symbolic_names = {0:'batch_size', 1: 'max_seq_len'} 
  torch.onnx.export(model,               # модель, которую будем экспортировать
                    dummy_inputs,                         # input модели
                    "output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx",   # путь сохранения модель
                    export_params=True,        
                    opset_version=11,          # версия ONNX, в который будем экспортировать модель
                    do_constant_folding=True,
                    input_names = ["input_ids","attention_mask","token_type_ids"],
                    output_names = ['output'],
                    dynamic_axes={'input_ids': symbolic_names,        #если у нас динамический размер input
                                  'attention_mask' : symbolic_names,
                                  'token_type_ids' : symbolic_names},
                    training=TrainingMode.EVAL
                    )

Пробуем запустить в ONNX и посмотреть время инференса

In [94]:
sess_options = nxrun.SessionOptions()
providers = [
    'CUDAExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx", sess_options, providers)

In [97]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [98]:
%%time
predict_onnx(text)

CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 18.2 ms


0

Считаем качество

In [25]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7783219952095606, recall: 0.7634240759240759, f1score: 0.7558935727560435


Посмотрим на вес модели

In [26]:
!du -shc output/onnx_transforms/*

679M	output/onnx_transforms/rubert-base-cased-sentiment.onnx
679M	output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx
1.4G	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала на порядок выше, метрики качества не изменились

2. Есть библиотека transforms для трансформеров, [где все почти из коробки](https://huggingface.co/docs/transformers/serialization)

In [27]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [28]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [29]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model.to('cpu'),
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert-base-cased-sentiment.onnx"),
        opset=11)

Если проверить скорость инференса и качество, получим то же самое

In [30]:
model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment.onnx", sess_options, providers)

In [31]:
%%time
predict_onnx(text)

CPU times: user 15.5 ms, sys: 2.75 ms, total: 18.2 ms
Wall time: 18 ms


2

In [32]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7783219952095606, recall: 0.7634240759240759, f1score: 0.7558935727560435


# TorchScript

TorchScript — инструмент, который позволяет с помощью пары строк кода и нескольких щелчков мыши сделать из пайплайна на питоне отчуждаемое решение, которое можно встроить в систему на C++. А еще она будет на python работать быстрее из-за jit компиляции. В библиотеке transformers так же [есть почти из коробки](https://huggingface.co/docs/transformers/serialization#torchscript)

In [33]:
tokenizer_torchscript = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment', torchscript = True)
model_torchscript = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, torchscript=True).to(device)

In [34]:
dummy_input0 = torch.randint(1, 224, (1,512)).to(device)
dummy_input1 = torch.randint(0, 1, (1,512 )).to(device)
dummy_input2 =  torch.randint(0, 1, (1,512 )).to(device)
traced_model = torch.jit.trace(model_torchscript, [x.clone().detach() for x in dummy_inputs])

In [35]:
!mkdir -p output/torchscript

In [36]:
torch.jit.save(traced_model, "output/torchscript/rubert-base-cased-sentiment_traced.pt")

Пробуем загрузить и предиктить

In [37]:
@torch.no_grad()
def predict_torchscript(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = traced_model(**inputs)[0]
    predicted = torch.nn.functional.softmax(outputs, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [38]:
%%time
predict_torchscript(text)

CPU times: user 4 s, sys: 104 ms, total: 4.11 s
Wall time: 4.6 s


2

Считаем качество модели

In [39]:
predictions = [predict_torchscript(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7783219952095606, recall: 0.7634240759240759, f1score: 0.7558935727560435


 Посмотрим на вес модели

In [40]:
!du -shc output/torchscript/*

679M	output/torchscript/rubert-base-cased-sentiment_traced.pt
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала немного выше, метрики качества не изменились. Лучше, чем ничего. А вообще надо а с++ запускать, чтобы увидеть адекватный результат

# Прунинг модели

Model Pruning — обрезание избыточных частей сети для ускорения инференса без потери точности. Наглядно — откуда, сколько и как можно вырезать.

Есть очень много способов прунинга моделей, но здесь мы рассмотрим способ прунинга attention слоев

Есть два варианта, как прунить модель.

1 вариант - делать через torch.nn.utils.prune. В качестве примера есть данный [ноутбук](https://github.com/Huffon/nlp-various-tutorials/blob/master/pruning-bert.ipynb)

2 вариант - библиотека [nn_pruning](https://github.com/huggingface/nn_pruning) от HuggingFace


[Ссылка](https://aclanthology.org/2020.repl4nlp-1.18.pdf) на почитать про прунинг модели BERT

Запруним encoder слои

In [45]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [61]:
pruned_model = model.to(device)

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (pruned_model.bert.encoder.layer[i].attention.self.key, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.query, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.RandomUnstructured,
    amount=0.9
)
for p in parameters_to_prune:
  prune.remove(p[0], 'weight')

Выведем, что получилось

In [62]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0)

    denominator += pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 99.00%
Sparsity in Layer 1-th query weightt: 99.02%
Sparsity in Layer 1-th value weight: 99.02%

Sparsity in Layer 2-th key weight: 99.00%
Sparsity in Layer 2-th query weightt: 98.99%
Sparsity in Layer 2-th value weight: 99.00%

Sparsity in Layer 3-th key weight: 99.01%
Sparsity in Layer 3-th query weightt: 99.00%
Sparsity in Layer 3-th value weight: 99.00%

Sparsity in Layer 4-th key weight: 99.02%
Sparsity in Layer 4-th query weightt: 98.98%
Sparsity in Layer 4-th value weight: 98.99%

Sparsity in Layer 5-th key weight: 98.99%
Sparsity in Layer 5-th query weightt: 98.99%
Sparsity in Layer 5-th value weight: 99.01%

Sparsity in Layer 6-th key weight: 99.02%
Sparsity in Layer 6-th query weightt: 99.00%
Sparsity in Layer 6-th value weight: 99.01%

Sparsity in Layer 7-th key weight: 98.98%
Sparsity in Layer 7-th query weightt: 99.00%
Sparsity in Layer 7-th value weight: 99.01%

Sparsity in Layer 8-th key weight: 99.00%
Sparsity in Layer 8-th query weigh

Предиктим на запруненной модели

In [63]:
@torch.no_grad()
def predict_pruned(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = pruned_model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [72]:
%%time
predict_pruned(text)

CPU times: user 25.8 ms, sys: 0 ns, total: 25.8 ms
Wall time: 27.4 ms


0

In [56]:
!mkdir -p output/pruning

In [73]:
torch.save(pruned_model, 'output/pruning/rubert-base-cased-sentiment_pruned.pt')

Посчитаем качество модели

In [74]:
predictions = [predict_pruned(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.112, recall: 0.3333333333333333, f1score: 0.16766467065868262


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 Посмотрим на вес модели

In [77]:
!du -shc output/pruning/*

679M	output/pruning/rubert-base-cased-sentiment_pruned.pt
679M	total


Вывод - вес модели не поменялся, качество упало чуть-чуть по сравнению с оригинальной моделью, но не критично. Модель стала работать быстрее, но ценой небольшой потери качества

# Квантизация

Квантизация означает уменьшение численной точности весов модели. Один из популярных методов — k-means квантизация. Имея веса модели в матрице W с десятичными числами, веса кластеризуются с помощью k-means в N кластеров. Затем матрица W трансформируется в матрицу целых чисел от 1 до N, каждое из которых является указателем к центру кластера. Так можно сжать каждый элемент изначальной матрицы из 32-битного десятичного числа в log(N)-битные целые числа.

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)

Динамическая квантизация не требует ничего, поэтому она самая простая

In [78]:
!mkdir -p output/quantization

Можно квантизировать модель через библиотеку onnxruntime 

In [109]:
model_fp32 = 'output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx'
model_quant = 'output/quantization/rubert-base-cased-sentiment.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[MatMul_68]
Ignore MatMul due to non constant B: /[MatMul_73]
Ignore MatMul due to non constant B: /[MatMul_162]
Ignore MatMul due to non constant B: /[MatMul_167]
Ignore MatMul due to non constant B: /[MatMul_256]
Ignore MatMul due to non constant B: /[MatMul_261]
Ignore MatMul due to non constant B: /[MatMul_350]
Ignore MatMul due to non constant B: /[MatMul_355]
Ignore MatMul due to non constant B: /[MatMul_444]
Ignore MatMul due to non constant B: /[MatMul_449]
Ignore MatMul due to non constant B: /[MatMul_538]
Ignore MatMul due to non constant B: /[MatMul_543]
Ignore MatMul due to non constant B: /[MatMul_632]
Ignore MatMul due to non constant B: /[MatMul_637]
Ignore MatMul due to non constant B: /[MatMul_726]
Ignore MatMul due to non constant B: /[MatMul_731]
Ignore MatMul due to non constant B: /[MatMul_820]
Ignore MatMul due to non constant B: /[MatMul_825]
Ignore MatMul due to non constant B: /[MatMul_914]
Ignore MatMul due to non constant

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [110]:
sess_options = nxrun.SessionOptions()
providers = [
    'CUDAExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx", sess_options, providers)

In [112]:
%%time
predict_onnx(text)

CPU times: user 71.2 ms, sys: 884 µs, total: 72.1 ms
Wall time: 75.7 ms


2

Считаем качество

In [113]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7725823838727065, recall: 0.7604929792429793, f1score: 0.7525480092127617


Можно так же через библиотеку optimum от transformers

In [105]:
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained("blanchefort/rubert-base-cased-sentiment", feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert-base-cased-sentiment.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx')

Посмотрим на вес модели

In [ ]:
!du -shc output/quantization/*

436M	output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx
679M	output/quantization/rubert-base-cased-sentiment.onnx
171M	output/quantization/rubert-base-cased-sentiment.quant.onnx
1.3G	total


Вывод:  по сравнению с моделью в формате ONNX скорость инференса модели стала  выше, метрики качества немного просели.Вес модели не уменьшился